<a href="https://colab.research.google.com/github/eeroolli/SalsaAnnotation/blob/development/data/Running_OpenPose_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running preprocessing with OpenPose

This works in Colab and contains most of the functions needed.

Single file to run

*   VideoProcessing.ipynb
*   AnnotationFile.ipynb
*  Parsing-Openpose-Annotation.ipynb   


In [ ]:

# libraries needed from outside
import os
from os import mkdir
from os.path import exists, join, basename, splitext
import sys
import glob                       # help for finding files
from re import split              # regular expression string splitter
import subprocess                 #subprocess wraps around regular os commands
import shutil                     # used for copying files in the os.

from google.colab import drive
drive.mount('/content/gdrive')
 
if os.path.isdir('/content/gdrive/MyDrive'):     # Ana
  google_drive_path = '/content/gdrive/MyDrive'
if os.path.isdir('/content/gdrive/MyDrive/DSR'):  # Eero. ipython can deal with MyDrive, 
  # but some bash commands require the space in "My Drive" to work.  and some fail.
  # I believe this has to do with what kind of " " '' one is using, as they must not be mixed. 
  google_drive_path = '/content/gdrive/MyDrive' 
else:
  print("There is a problem google_drive_path.")

root_path   = os.path.join(google_drive_path ,  "DanceApp")  
script_path = "/content/gdrive/MyDrive/2021-DSR-Porfolio/python_scripts"
#one cannot add items to sys.path that have a space in them. so script path must
#the same for Ana and Eero.  Eero has now added a link from his DSR to Ana's 2021-DSR-Porfolio/
#script_path = os.path.join(google_drive_path , "2021-DSR-Porfolio/python_scripts")
print("Using ", root_path, " as root_path")
# here = os.getcwd()
# print(here)
# print(sys.path)
# sys.path = ['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']



## Build OpenPose

In [ ]:

# Path and variables to build OpenPose
git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):    ## This saves Lots of time when rerunning the whole notebook.
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
# !ffprobe -v 0 -of csv=p=0 -select_streams v:0 -show_entries stream=r_frame_rate $video 
# Function to run OpenPose TODO, add this function to python script





## Define Functions

### check_path() &  stop_if_no_path()

In [ ]:
# TODO: move this to /utils ?

def check_path(x):
  if os.path.isdir(x):
    print("Using", x )
  else:
    print("Problem: There was no ", x)  # No error keeps running

def stop_if_no_path(x):
  if os.path.isdir(x):
    print("Using", x )
  else:
    raise Exception("Problem: There was no ", x)  # Throws an error and stops

### resize_video()

In [ ]:
def resize_video(new_height, video_in, clip_name, src_folder):
    video_name, video_ext = splitext(clip_name)
    print("\n ################################ \n")
    print("#     Resizing the video:     ")
    print("clip_name: ", clip_name)
    print("video_name: ", video_name, " extention: ", video_ext)
    
    if os.path.isfile(video_in):
      print("Found the input video_in")
    else:
      print("The input video does not exist.")
      # raise Exception("No input video file")

    print("new hight will be: ", new_height)
    resize_output_dir = os.path.join(root_path, src_folder ,"h" + str(new_height))
    video_out = resize_output_dir + "/" + video_name + "_h" + str(new_height) + video_ext
    print("the output video will be: ", video_out)

    if not os.path.isdir(resize_output_dir):
      os.mkdir(resize_output_dir)
      print("Created a folder for resized video output:", src_folder, "/h",str(new_height) )
    else:
      print("Folder for resized video output:", src_folder, "/h",str(new_height) )
    
    if os.path.isfile(video_out):
      print("The video has been resized previously")
    else:
      print("The video is being resized, which takes often several minutes.")
      !ffmpeg -i $video_in -vf scale=-2:$new_height $video_out    # the -2 ensures that width is a even number as required
    return video_out

### load_video_run_openpose()

In [ ]:
def load_video_run_openpose(video):
  #requires that video_id and connected variables are set.   
  print("\n ################################ \n")
  print("#     Run OpenPose:     ")
  # Check for the speed of the video
  frames_per_s = default_fps            # if everything fails there is a default value
  if not os.path.isfile(video):
    print("There video is missing: \n ", video)
  else:
    print("OpenPose will now analyse \n", video)

  print(os.getcwd())
  text = !ffprobe -v 0 -of csv=p=0 -select_streams v:0 -show_entries stream=r_frame_rate $video 
  print(text)
  #frames, seconds = text.split("/")
  frames, seconds = text[0].split("/")
  frames_per_s = int(frames) / int(seconds)
  
  print("Number of frames per second: ", frames_per_s)
  print("\n")
  if frames_per_s > 20:
    multiplikator_for_images_per_second = frames_per_s / save_frames_per_second
  else:
    multiplikator_for_images_per_second = default_fps / save_frames_per_second
  
  # cut the parts of video that we cannot use
  # !ffmpeg -y -loglevel info -i "input_original/youtube.mp4" -ss $start_mmss -t $duration "input_original/video.mp4"
  
  # detect poses
  print(f"Video for Openpose {video}") 
  op_video = output_op_dir + "/openpose.avi"
  op_json = output_op_dir + "/json"

  if not os.path.exists(op_json): 
    !mkdir $op_json
  
  if skeleton_on_black_background==True: 
    !cd openpose && ./build/examples/openpose/openpose.bin --video $video --write_json --number_people_max 1 $op_json --display 0 --disable_blending  --write_video $op_video
  elif skeleton_on_black_background==False: 
    !cd openpose && ./build/examples/openpose/openpose.bin --video $video --write_json --number_people_max 1 $op_json --display 0  --write_video $op_video
  else:
    raise Exception("You need to set the variable skeleton_on_black_background") 

  
  # convert the result into MP4
  print(" ")
  print("============================== ")
  print("Converting the stickfigure video to mp4 ")
  print(" ")

  op_video_mp4 = output_op_dir + "/openpose.mp4"

  if os.path.isfile(op_video): 
    !ffmpeg -y -loglevel info -i $op_video $op_video_mp4 #convert avi til mp4
    os.remove(op_video)  # remove avi
  else:
    print("Something went wrong, as there was no output video from Openpose.")
    print("the output_op_dir contains:")
    print(os.listdir(output_op_dir))

  # print(" ")
  # print("Creating single frame images ")
  # print("============================== ")
  # print(" ")

  ## Get only a selection of frames for labeling
  ## TODO: the numbering should match the numbering of the json files
  # save_frames_per_second = save_frames_per_second 
  # Optional to get the frames with the stick figures
  # !ffmpeg -y -loglevel info -i "output_op/$clip_name/openpose.mp4" -s $size_of_single_frame -r $save_frames_per_second -f image2 "output_op/$clip_name/frame-%06d.jpeg" 

  print(" ")
  print("============================== ")  
# print(video)
# text = subprocess.getoutput('ffprobe -v 0 -of csv=p=0 -select_streams v:0 -show_entries stream=r_frame_rate $video') 
# text.split("/")
#frames, seconds = 
# frames_per_s = int(frames) / int(seconds)
# frames_per_s  




### what does this do?  Can it be removed?

In [ ]:
check_path(script_path)
print(os.listdir(script_path))

if not script_path in sys.path:    #otherwise will add anew with every run of script.
   sys.path.append(script_path)
   print("Added script_path to search path.")

# from videoprocessing import delete_outputs, load_video_run_openpose
import VideoProcessing as vp

# here = os.getcwd()
# print(here)
# print(sys.path)
# sys.path = ['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']

### delete_outputs

In [ ]:
def delete_outputs(video_id, root_path, output_dir):
    # delete output_files from Openpose if they were already created
    print("\n ################################ \n")
    print("Deleting old output files from the ", output_dir, video_id, " folder")
    # create a list of files to delete
    delete_files = glob.glob(os.path.join(root_path, output_dir, video_id, "json", "*.json"))
    delete_files.extend(glob.glob(os.path.join(root_path, output_dir, video_id, "*.mp4")))
    delete_files.extend(glob.glob(os.path.join(root_path, output_dir, video_id, "*.MOV")))
    # delete_files.extend(glob.glob(os.path.join(root_path, "output_op", clip_name, "*.avi")))
    # print("avi: ", delete_files[-1:])
    [os.remove(x) for x in delete_files]

    print("\n Now the output folder contains only these files:")
    print(glob.glob(os.path.join(root_path, output_dir, video_id, "json", "*.json")))

### rename_json()

In [ ]:
def rename_json(video_id, root_path, output_dir):
    print("\n ################################ \n")
    print("#     Rename JSON files:     ")
    ## Better names for JSON
    # find all json files and change the name
    # os.chdir(original_working_dir)
    print(os.getcwd())
    paths_json = glob.glob(os.path.join(root_path, output_dir, video_id, "json", "*.json"))
    paths_json = sorted(paths_json, reverse=True)

    print("there are ", len(paths_json), " json files to rename")
    if len(paths_json)>0:
      for old_file in paths_json:
          frame_number = old_file[-21:-15]  # counting backwards is safest, because folder names change
          # print(f"Frame {frame_number}")
          # print(f"Old name {old_file}")
          new_file_name = os.path.join(root_path, output_dir, video_id, "json") + "/frame-" + frame_number + ".json"
          # print(f"New name {new_file_name}")
          os.rename(old_file, new_file_name)

      print(" ")
      print("All json are now renamed.")
    else:
      print("nothing to be done.")  
    print("============================== ")

    # delete old names
    # [os.remove(x) for x in paths_json]
    print(" ")

In [ ]:
# ??vp.rename_json() 
os.path.isfile("/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_ThomasW_16012122_choreo1.mp4")

### play_video_in_colab

In [ ]:
def play_video_in_colab(video_show):
  #video_show should be a full path.
  # this only plays compressed files!
  from IPython.display import HTML
  from base64 import b64encode
  if os.path.isfile(video_show):
      print("The videofile to be shown does exist.")
      print("This player is not able to play large uncompressed videos.")
  else:
      print("The videofile to be shown does NOT exist.")

  mp4 = open(video_show,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  HTML("""
  <video width=300 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

### Watch single video

In [ ]:
video_show = "/content/gdrive/MyDrive/DanceApp/1P_input_original/h920/1p_ThomasW_girl_16012022_choreo1_h920.mp4"
# video_show = "/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_ThomasW_girl_16012022_choreo1.mp4"
# play_video_in_colab(video_show=video_show)



# Main Part

## Variables = controlling the run

In [ ]:
# Global variables with Paths for all the functions
size_of_single_frame = "640x360"
save_frames_per_second = 5  # the number of jpgs per second TODO: remove?
default_fps = 30     # the default frames per second of the videos, 
                     # which often is wrong.
                     # IF we use timecodes it
                     # needs to be corrected after the download
                     # so that the json and jpg files match each other 
                     
video_size = "h920"        # used to file and folder names
output_dir = "output_op_1p"
input_dir = "1P_input_original"
skeleton_on_black_background = True

# The names of the videos to be processed.
# these can NOT have spaces in them. Rename with underscore".
"""

video_list = [ 
 '1p_Artur_05122021_chreo1.mp4',
 '1p_Earl_05122021_choreo1_flipped_lr.mp4',
 '1p_Eero_1_05122021_choreo1.mp4',
 '1p_Eero_4_05122021_choreo1.mp4',
 '1p_Eero_6_05122021_choreo1.mp4',
 '1p_Eero_7_05122021_choreo1.mp4',
 '1p_Dirk_choreo1_04.12.2021.mp4',
 '1p_Heidi_07122021_choreo1_-turns.MOV',
 '1p_JonathanB_05122021_choreo1.MOV',
 '1P-Person1-whole.mp4',
 '1p_Raquelinha_05122021_choreo1.mp4',
 '1p_sarah_salsa_basics_choreo1.mp4',
 '1p_Stef_1_05122021_Choreo1.mp4',
 '1p_Stef_2_05122021_Choreo1.mp4',
 '1p_Stef_3_05122021_chroeo1.mp4',
]
"""

#third list of videos
"""video_list = [ 
              "1p_abtin_1_choreo1.mp4",
              "1p_abtin_2_choreo1.mp4",
              "1p_Vassil_2_11122021_Choreo1.mp4",
]
"""
#/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_ThomasW_16012122_choreo1.mp4
#fourth list of videos
video_list = ["1p_ThomasW_16012022_choreo1.mp4"]
              # "1p_Daniele_16012022_choreo1.mp4",
              # "1p_20211216_Francesca_Zeni.mp4",
              # "1p_ThomasW_girl_16012022_choreo1.mp4",
                           



In [ ]:
### Variables that are calculated from the ones above.
new_height = "".join([chr for chr in video_size if chr.isdigit()]) #used to resize          
input_video_fullsize_dir = os.path.join(root_path, input_dir) 
input_video_resized_dir = os.path.join(root_path, input_dir, video_size)
check_path(input_video_fullsize_dir)
check_path(input_video_resized_dir)


In [ ]:
check_path(root_path)
check_path(script_path)


## Processing the videos

In [ ]:

print("\n ################################ \n")

for i in range(len(video_list)):
  clip_name = video_list[i]
  video_id = splitext(video_list[i])[0]
  video_in = os.path.join(input_video_fullsize_dir, clip_name)
  video_name, video_ext = splitext(video_in)
  video_resized = input_video_resized_dir + "/" + video_id + '_' + video_size + video_ext 
  print("video fullsize: ", video_in)
  print("video_resized: ", video_resized)
  print(video_ext)

  # Video Processing and OpenPose  
  #from pathlib import Path as P
  output_main = os.path.join(root_path, output_dir)
  if not os.path.isdir(output_main): 
    os.makedirs(output_main, exist_ok=True)
  
  output_op_dir = os.path.join(output_main, video_id)
  if not os.path.isdir(output_op_dir): 
    os.makedirs(output_op_dir, exist_ok=True) 
  
  print("Processing video", clip_name)
  
  video_resized = resize_video(new_height = new_height, 
                               video_in=video_in, 
                               clip_name=clip_name, 
                               src_folder=input_dir)
  
  delete_outputs(video_id=video_id, root_path=root_path, output_dir=output_dir)
  load_video_run_openpose(video=video_resized)
  rename_json(video_id, root_path=root_path, output_dir=output_dir)

  # Creating the annotation file
  # TODO: make os independent. use shutil.copy(src, dst) and os.path() 
  # !cp $script_path/AnnotationFile.py .
  # !python AnnotationFile.py $output_op_dir
  
  # Parsing JSON and adding information from annotation
  # Anot_file = output_op_dir + "/Annotation.json" 
  # !cp $script_path/Parsing-Openpose-Annotation.py .
  # !python Parsing-Openpose-Annotation.py $video_id $output_op_dir $Anot_file



